![Alt text](https://monosnap.com/image/XWHOBpayEzmvdsR3ieN34UKGIo2NQu.png) ![Alt text](https://monosnap.com/image/ZI0ytTJUYBvLXOZOQSNwHhBb5cZxx5.png)


In [ ]:
def do_keyEventLoop():
    global img,img2,output,value,mask, rect,frame_or_mask, mouseCallBacker,filename,quitflag
    # キーイベントループ
    while(1):
    
        k = cv2.waitKey(1) # 1milisecond 入力を受け付ける　　0にすると無限に待ってしまう

        if k == 27 or k == ord('q')  or  k ==ord('`'):         # 終了は ESC
            quitflag = True
            break
            
        elif k == ord('0'): # 背景領域の指定
            # print(" 背景領域を指定 \n")
            value = DRAW_BG
        elif k == ord('1'): # 対象の指定
            # print(" 切り出し対象領域を指定 \n")
            value = DRAW_FG
        elif k == ord('2'): # 背景かも知れない領域の指定
            # print(" 背景かも知れない領域の指定 \n")
            value = DRAW_PR_BG
        elif k == ord('3'): # 前景かもしれない領域の指定
            # print(" 前景かもしれない領域の指定 \n")
            value = DRAW_PR_FG
            
        elif k == ord('+'):
            mouseCallBacker.thicknessUp()

        elif k == ord('-'):
            mouseCallBacker.thicknessDown()
            
        elif k == ord('9'): # 90度回転
            # print(" 回転します\n")
            img = img.transpose(1,0,2)[::-1,:,:]
            img2=img2.transpose(1,0,2)[::-1,:,:]
            mask=mask.transpose(1,0)[::-1,:]
            output=output.transpose(1,0,2)[::-1,:,:]
            width , height = img.shape[1], img.shape[0]
            rect= (rect[1],height-rect[0]-rect[2],rect[3],rect[2])
            cv2.imshow('output',redmasked(output))
            cv2.imshow('input',img)
            
        elif k == ord('s'): # 画像の保存
            bar = np.zeros((img.shape[0],5,3),np.uint8)
            res = np.hstack((img2,bar,img,bar,output))
            
            print("抽出結果を保存するパスを選んで下さい（拡張子は不要）")
            savepath = saveFilePath(filename)
            
            _ret,bw = cv2.threshold(cv2.cvtColor(output,cv2.COLOR_BGR2GRAY),1,255,cv2.THRESH_BINARY)
            alignedimg = rotateAndcut1(bw,ratio=4) # 主方向に合わせて回転した２値画像

            filepath, ext = os.path.splitext(savepath)
            cv2.imwrite(filepath+"Cut.png",output)
            cv2.imwrite(savepath,alignedimg)
            
            cv2.imwrite('grabcut_output.png',res)
            print("抽出結果は保存先:{}に、\n, それとは別に合成画像を grabcut_output.png に結果を保存しました.\n".format(savepath+".png"))
            quitflag = False
            break
            
        elif k == ord('r'): # reset everything
            print("リセット \n")
            mouseCallBacker.init()
            img = img2 .copy()  # 画像を復元
            mask = np.zeros(img.shape[:2],np.uint8) # mask initialized to PR_BG
            output = np.zeros(img.shape,np.uint8)           # 出力画像用
            mouseCallBacker.thickness = 3
            
        elif k == 13 : #  Enter キー  セグメンテーションの実行
            # print("セグメンテーションの実行中。新しいメッセージが表示されるまでお待ち下さい。 \n")
            if (frame_or_mask == 0):         # grabcut with rect
                bgdmodel = np.zeros((1,65),np.float64)
                fgdmodel = np.zeros((1,65),np.float64)
                mask = mask.copy()
                cv2.grabCut(img2,mask,rect,bgdmodel,fgdmodel,1,cv2.GC_INIT_WITH_RECT)
                frame_or_mask = 1
            elif frame_or_mask == 1:         # grabcut with mask
                bgdmodel = np.zeros((1,65),np.float64)
                fgdmodel = np.zeros((1,65),np.float64)
                cv2.grabCut(img2,mask,rect,bgdmodel,fgdmodel,1,cv2.GC_INIT_WITH_MASK)
            # print(" 抽出がうまくいっていない場合は、手動でタッチアップしてから再度 N  を押して下さい。\n ０、２　背景領域の指定、１，３ 抽出対象領域の指定 \n")
            
        mask2 = np.where((mask==1) + (mask==3),255,0).astype('uint8')
        output = cv2.bitwise_and(img2,img2,mask=mask2)
        if usecolor == 1:

            _ret,bw = cv2.threshold(cv2.cvtColor(output,cv2.COLOR_BGR2GRAY),1,255,cv2.THRESH_BINARY)
            output =  bw

In [40]:
class myMouse:
    def __init__(self, windowname):
        self.init()
        cv2.setMouseCallback(windowname, self.callBack, None)
        
    def init(self):
        global value,mask,framing,framed,drawing,rect,frame_or_mask
        # setting up flags
        rect = (0,0,1,1)
        drawing = False         # 描画モードオン
        framing = False           # 選択枠設定中
        framed = False       # 枠設定は完了している
        frame_or_mask = 100      # flag for selecting rect or mask mode
        value = DRAW_FG         # drawing initialized to FG
        self.thickness = 3           # ブラシサイズ
    
    def thicknessUp(self):
        self.thickness +=1
        
    def thicknessDown(self):
        if self.thickness > 0:
            self.thickness -=1
        
    def callBack(self, event, x, y, flags, param=None) :
        global img,img2,output,value,mask,framing,framed,drawing,rect,frame_or_mask
        # フレーム設定フェーズの処理
        if framed == False:
            if event == cv2.EVENT_LBUTTONDOWN:
                self.lx,self.ly = x,y
                framing = True # 矩形描画モードオン
            elif event == cv2.EVENT_MOUSEMOVE:  
                if framing == True :                      
                    img = img2.copy()
                    cv2.rectangle(img,(self.lx,self.ly),(x,y),BLUE,2)
                    rect = (min(self.lx,x),min(self.ly,y),abs(self.lx-x),abs(self.ly-y))  
            elif event == cv2.EVENT_LBUTTONUP:
                framing = False
                tmps = abs(self.lx-x)*abs(self.ly-y)  # 指定矩形の面積
                if tmps > MINRECTSIZE:
                    framed = True
                    cv2.rectangle(img,(self.lx,self.ly),(x,y),BLUE,2)
                    rect = (min(self.lx,x),min(self.ly,y),abs(self.lx-x),abs(self.ly-y))
                    frame_or_mask = 0
                    print(" Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい \n")
        else: # 枠指定がすでに済んでいる場合
            if drawing == True:
                cv2.circle(img,(x,y),self.thickness,value['color'],-1)
                cv2.circle(mask,(x,y),self.thickness,value['val'],-1)
                if event == cv2.EVENT_LBUTTONUP:
                    drawing = False
            else:
                if event == cv2.EVENT_LBUTTONDOWN:
                    drawing = True
                    
        cv2.imshow('output',redmasked(output))
        cv2.imshow('input',img)

In [41]:
# 画像サイズの縦か横の大きい方が指定サイズになるようにリサイズする。
def resize(img, size=512):
    maxsize = np.max(np.array(img.shape[:2]))
    height = 2*size*img.shape[0]//maxsize
    width = 2*size*img.shape[1]//maxsize
    keepimg = np.zeros((height+80, width+80,3),np.uint8)
    keepimg[40:40+height,40:40+width]=cv2.resize(img,(width,height))
    output = np.zeros((keepimg.shape[0]//2,keepimg.shape[1]//2,3),np.uint8)
    output[20:20+height//2,20:20+width//2]=cv2.resize(img,(width//2,height//2))
    return output,keepimg

def redmasked(img):
    ones = np.ones(img.shape[:2],np.uint8)
    zeros = np.zeros(img.shape[:2],np.uint8)
    ret,img2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV)
    b,g,r = cv2.split(img2)
    red = cv2.bitwise_and(ones,b)
    red = cv2.bitwise_and(red,g)
    red = cv2.bitwise_and(red,r)
    red = cv2.merge((zeros,zeros,red*128))
    red = cv2.bitwise_or(red,img)
    return red


In [42]:
# 余分なマージンをカットする関数
# 余分なマージンをカットした画像を返す関数． オブジェクトサイズ/ratio だけマージンを残す 
# 対象とする画像はシルエットが白のシルエット画像に限定
def cutmargin(img,ratio=4,usecolor=False) :
    global usecolor
    usecolor=usecolor
    x,y,w,h = cv2.boundingRect(img)
    x2 = int(w/ratio)
    y2 = int(h/ratio)
    shp = (h+2*y2,w+2*x2)
    outimg = np.zeros(shp, dtype=np.uint8) 
    outimg[y2:y2+h,x2:x2+w]=img[y:y+h,x:x+w]
    return outimg

#  使用例
'''
# グレイ画像の場合，もしくは結果をグレー化したい場合
src = cv2.imread(imgfile,0)
cutimg = cutmargin(src, ratio=4)   
plotimg(cutimg)
'''

def imagerotation(img, deg,cx= -1, cy=-1 , ratio=4):  # deg 回転角，　(cx,cy) 回転中心  ratio
    if cx < 0:
        cx = img.shape[1]/2
        cy = img.shape[0]/2
    rows,cols = img.shape[:2] 
    need = int(np.sqrt(rows**2+cols**2)) # 描画エリアを十分確保するために対角の長さを計算
    xoff = int((need-cols)/2)
    yoff = int((need-rows)/2)
    tmpimg = np.zeros((need,need),dtype=np.uint8) # 描画用画像エリア
    tmpimg[yoff:yoff+rows,xoff:xoff+cols]=img # シルエット画像をコピー
    mat = cv2.getRotationMatrix2D((cx+xoff,cy+yoff), deg, 1.0)
    outimg = cv2.warpAffine(tmpimg, mat, (0,0))
    outimg = cutmargin(outimg,ratio)
    return outimg

# 回転を考慮した矩形でカットし，それを水平に向けた上でマージンをカット
def rotateAndcut1(img,ratio=4) :
    cntimg = np.copy(img)
    _img,contours,hierarchy = cv2.findContours(cntimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    radishcon = contours[0] # ダイコンの輪郭線
    para_reps = 0.01 # Sufficient accuracy for the radius (distance between the coordinate origin and the line).
    para_aeps = 0.01 # Sufficient accuracy for the angle. 0.01 would be a good default value for reps and aeps.
    # cv2.DIST_L2 は あてはめ誤差の評価基準　https://goo.gl/i0UQag
    [vx,vy,x0,y0] = cv2.fitLine(radishcon, cv2.DIST_L2,0,para_reps,para_aeps)        
    # deg = math.atan2(vy,vx)*180/np.pi - 90 # 必要な回転角の計算
    deg = 0
    outimg = imagerotation(img,deg,x0,y0, ratio)
    return outimg

In [43]:
import tkinter
import tkinter.filedialog
import tkinter.messagebox
%gui tk
root=tkinter.Tk()
root.withdraw()

def readFilePath():
    fTyp=[('画像ファイルの選択',['jpg','png'])]
    readpath=tkinter.filedialog.askopenfilename(filetypes=fTyp,initialdir = './一軍')
    print("処理対象ファイル",readpath)
    return readpath

def saveFilePath(filename):                         
    savepath = tkinter.filedialog.asksaveasfilename(initialdir = "シルエット/",initialfile=filename,filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
    return savepath

In [ ]:
while True:
    cutImage(usecolor=True)
    if quitflag:
        break
    


# グレイ画像の場合，もしくは結果をグレー化したい場合
src = cv2.imread(imgfile,0)
cutimg = cutmargin(src, ratio=4)   
plotimg(cutimg)

画像ファイルを選んで下さい
処理対象ファイル /Users/dan/Desktop/Yumiko/一軍/17Cylindric/17tanabe2o04_l.jpg
 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

抽出結果を保存するパスを選んで下さい（拡張子は不要）
抽出結果は保存先:/Users/dan/Desktop/Yumiko/シルエット/17Cylindric/17tanabe2o04_l.jpg.pngに、
, それとは別に合成画像を grabcut_output.png に結果を保存しました.


# グレイ画像の場合，もしくは結果をグレー化したい場合
src = cv2.imread(imgfile,0)
cutimg = cutmargin(src, ratio=4)   
plotimg(cutimg)

画像ファイルを選んで下さい
処理対象ファイル /Users/dan/Desktop/Yumiko/一軍/17Cylindric/17tanabe2o07_l.jpg
 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

抽出結果を保存するパスを選んで下さい（拡張子は不要）
抽出結果は保存先:/Users/dan/Desktop/Yumiko/シルエット/17Cylindric/17tanabe2o07_l.jpg.pngに、
, それとは別に合成画像を grabcut_output.png に結果を保存しました.


# グレイ画像の場合，もしくは結果をグレー化したい場合
src = cv2.imread(imgfile,0)
cutimg = cutmargin(src, ratio=4)   
plotimg(cutimg)

画像ファイルを選んで下さい
処理対象ファイル /Users/dan/Desktop/Yumiko/一軍/17Cylindric/17tanabe2o09_l.jpg
 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

抽出結果を

処理対象ファイル /Users/dan/Desktop/Yumiko/一軍/17Spheric/17marumi2o04_l.jpg
 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

リセット 

 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

リセット 

 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

リセット 

 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

リセット 

 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

抽出結果を保存するパスを選んで下さい（拡張子は不要）
抽出結果は保存先:/Users/dan/Desktop/Yumiko/シルエット/17Spheric/17marumi2o04_l.jpg.pngに、
, それとは別に合成画像を grabcut_output.png に結果を保存しました.


# グレイ画像の場合，もしくは結果をグレー化したい場合
src = cv2.imread(imgfile,0)
cutimg = cutmargin(src, ratio=4)   
plotimg(cutimg)

画像ファイルを選んで下さい
処理対象ファイル /Users/dan/Desktop/Yumiko/一軍/17Spheric/17tyokei1o05_l.jpg
 Enterキーを押せば抽出を始めます。終わるまでしばらくお待ち下さい 

抽出結果を保存するパスを選んで下さい（拡張子は不要）
抽出結果は保存先:/Users/dan/Desktop/Yumiko/シルエット/17Spheric/17tyokei1o05_l.jpg.pngに、
, それとは別に合成画像を grabcut_output.png に結果を保存しました.


# グレイ画像の場合，もしくは結果をグレー化したい場合
src = cv2.imread(imgfile,0)
cutimg = cutmargin(src, ratio=4)   
plotimg(cutimg)

画像ファイルを選んで下さい
処理対象ファイル /Users/dan/Desktop/Yumiko/一軍/17Spheric/17yakumi1o05_l.j